# Workflow Execution Service Endpoints
Find the detailed GA4GH WES Specification [here](https://github.com/ga4gh/workflow-execution-service-schemas).

In [1]:
import requests
import json

wes_port = "6000"
ga4gh_base_url = "http://localhost:{}/ga4gh/{}/v1"
wes_base_url = ga4gh_base_url.format(wes_port,"wes")

service_info_path = "/service-info"
runs_path = "/runs"

def pretty_print_json(response):
    # pretty print JSON in blue color
    print("\033[38;2;8;75;138m"+json.dumps(response.json(), indent=4)+"\033[0m")
    
def print_head(text):
    # print in green color
    print("\033[38;2;8;138;75m"+text+"\033[0m")

## i. Get `/service-info`
**Retrieve information about this service**

In [2]:
http_method = "GET"
request_url = wes_base_url+service_info_path

print_head("{} request to {}".format(http_method, request_url))

# GET request to service-info endpoint
wes_service_info_resp = requests.request(http_method, request_url)

# print the response
pretty_print_json(wes_service_info_resp)

GET request to http://localhost:6000/ga4gh/wes/v1/service-info
{
    "id": "wes.starterkit.federatedgenomics.org",
    "name": "Federated Genomics WES service",
    "description": "Workflow Execution Service (WES) instance. Enables small-scale test workflows to be run over the web. Deployment of the GA4GH Starter Kit.",
    "contactUrl": "mailto:nobody@federatedgenomics.org",
    "documentationUrl": "https://apidocs.federatedgenomics.org/wes",
    "createdAt": "2022-07-10T09:00:00Z",
    "updatedAt": "2022-07-10T09:00:00Z",
    "environment": "development",
    "version": "1.0.0",
    "type": {
        "group": "org.ga4gh",
        "artifact": "wes",
        "version": "1.0.1"
    },
    "organization": {
        "name": "Federated Genomics",
        "url": "https://this-is-not-a-site.federatedgenomics.org"
    },
    "workflow_type_versions": {
        "WDL": [
            "1.0"
        ],
        "NEXTFLOW": [
            "21.04.0"
        ]
    },
    "workflow_engine_versions": {
 

## ii. List `/runs`
**Retrieves the list of runs**

In [3]:
http_method = "GET"
request_url = wes_base_url+runs_path

print_head("{} request to {}".format(http_method, request_url))

# GET request to /runs endpoint
wes_list_runs_resp = requests.request(http_method, request_url)

# print the response
pretty_print_json(wes_list_runs_resp)

GET request to http://localhost:6000/ga4gh/wes/v1/runs
{
    "runs": []
}


## iii. Post a Nextflow workflow
**Run `samtools head` on this file: `https://ga4gh-ismb-tutorial-2022.s3.us-east-2.amazonaws.com/data/1000genomes/cram/lowcov/HG00118.lowcoverage.downsampled.cram`**

In [4]:
http_method = "POST"
request_url = wes_base_url + runs_path

nextflow_workflow_url = "https://github.com/yash-puligundla/samtools-head-nf"
input_file = "https://ga4gh-ismb-tutorial-2022.s3.us-east-2.amazonaws.com/data/1000genomes/cram/lowcov/HG00118.lowcoverage.downsampled.cram"

data = {
    'workflow_type': 'NEXTFLOW',
    'workflow_type_version': '21.04.0',
    'workflow_url': nextflow_workflow_url,
    'workflow_params': f'{{"input":"{input_file}"}}'
}

print_head("{} request to {}".format(http_method, request_url))

# Post a Nextflow workflow
wes_post_workflow_response = requests.request(http_method, request_url, data = data)

# print the response
pretty_print_json(wes_post_workflow_response)

current_run_id = wes_post_workflow_response.json()["run_id"]

print_head("run_id = {}".format(current_run_id))

POST request to http://localhost:6000/ga4gh/wes/v1/runs
{
    "run_id": "3abe5a5c-077b-471c-9f8f-8b16134e9067"
}
run_id = 3abe5a5c-077b-471c-9f8f-8b16134e9067


### iii a. Monitor the run

In [5]:
http_method = "GET"
request_url = wes_base_url + runs_path + "/" + current_run_id

print_head("{} request to {}".format(http_method, request_url))

# Get request to /runs/{run_id}
monitor_run_response = requests.request(http_method, request_url)

# print the response
pretty_print_json(monitor_run_response)

GET request to http://localhost:6000/ga4gh/wes/v1/runs/3abe5a5c-077b-471c-9f8f-8b16134e9067
{
    "run_id": "3abe5a5c-077b-471c-9f8f-8b16134e9067",
    "request": {
        "workflow_params": {
            "input": "https://ga4gh-ismb-tutorial-2022.s3.us-east-2.amazonaws.com/data/1000genomes/cram/lowcov/HG00118.lowcoverage.downsampled.cram"
        },
        "workflow_type": "NEXTFLOW",
        "workflow_type_version": "21.04.0",
        "workflow_url": "https://github.com/yash-puligundla/samtools-head-nf"
    },
    "state": "COMPLETE",
    "run_log": {
        "name": "yash-puligundla/samtools-head-nf",
        "cmd": [
            "#!/bin/bash -ue",
            "samtools head https://ga4gh-ismb-tutorial-2022.s3.us-east-2.amazonaws.com/data/1000genomes/cram/lowcov/HG00118.lowcoverage.downsampled.cram > output_samtools_head.txt"
        ],
        "start_time": "2023-06-30T19:03:51Z",
        "end_time": "2023-06-30T19:04:02Z",
        "stdout": "http://localhost:6000/ga4gh/wes/v1/lo

## iv. Post a Nextflow workflow
**Run `samtools head` on the drs_object with DRS URI `drs://localhost:5000/8e18bfb64168994489bc9e7fda0acd4f`**

In [6]:
http_method = "POST"
request_url = wes_base_url+runs_path

nextflow_workflow_url = "https://github.com/yash-puligundla/samtools-head-nf"
input_file = "drs://localhost:5000/77b0f3d65271c4a0064ff7760828dd92"

data = {
    'workflow_type': 'NEXTFLOW',
    'workflow_type_version': '21.04.0',
    'workflow_url': nextflow_workflow_url,
    'workflow_params': f'{{"input":"{input_file}"}}'
}

print_head("{} request to {}".format(http_method, request_url))

# Post a Nextflow workflow
wes_post_workflow_response = requests.request(http_method, request_url, data = data)

# print the response
pretty_print_json(wes_post_workflow_response)

current_run_id = wes_post_workflow_response.json()["run_id"]

print_head("run_id = {}".format(current_run_id))

POST request to http://localhost:6000/ga4gh/wes/v1/runs
{
    "run_id": "21675abb-cdc1-4b34-a0de-c100f0cc2cda"
}
run_id = 21675abb-cdc1-4b34-a0de-c100f0cc2cda


### iv a. Monitor the run

In [7]:
http_method = "GET"
request_url = wes_base_url + runs_path + "/" + current_run_id

print_head("{} request to {}".format(http_method, request_url))

# Get request to /runs/{run_id}
monitor_run_response = requests.request(http_method, request_url)

# print the response
pretty_print_json(monitor_run_response)

GET request to http://localhost:6000/ga4gh/wes/v1/runs/21675abb-cdc1-4b34-a0de-c100f0cc2cda
{
    "run_id": "21675abb-cdc1-4b34-a0de-c100f0cc2cda",
    "request": {
        "workflow_params": {
            "input": "drs://localhost:5000/77b0f3d65271c4a0064ff7760828dd92"
        },
        "workflow_type": "NEXTFLOW",
        "workflow_type_version": "21.04.0",
        "workflow_url": "https://github.com/yash-puligundla/samtools-head-nf"
    },
    "state": "COMPLETE",
    "run_log": {
        "name": "yash-puligundla/samtools-head-nf",
        "cmd": [
            "#!/bin/bash -ue",
            "samtools head https://s3.us-east-2.amazonaws.com/ga4gh-ismb-tutorial-2022/data/1000genomes/cram/highcov/NA12842.final.2400kb.cram > output_samtools_head.txt"
        ],
        "start_time": "2023-06-30T19:04:54Z",
        "end_time": "2023-06-30T19:05:05Z",
        "stdout": "http://localhost:6000/ga4gh/wes/v1/logs/nextflow/stdout/21675abb-cdc1-4b34-a0de-c100f0cc2cda?workdirs=4f%2F944f61610928

## For more endpoints, find the detailed GA4GH WES Specification [here](https://github.com/ga4gh/workflow-execution-service-schemas).